# Validations

In [ ]:
using Plots
using Plots.PlotMeasures
using ModelingToolkit
using DifferentialEquations
using LaTeXStrings
using RetroSignalModel
import RetroSignalModel as rs

In [ ]:
@named sys = RtgMTK(rs.ONE_SIGNAL)
u0 = rs.resting_u0(sys)

ks = load_parameters("solution_rtgM4.csv")[1]
prob = SteadyStateProblem(sys, u0, ks)
# Warm up
sol = solve(prob, DynamicSS(Rodas5()));

In [ ]:
psmap = Dict(k => i for (i, k) in enumerate(parameters(sys)))

## Bmh partial knockout

Source: https://github.com/ntumitolab/MitoRetroDynamics/blob/main/validation.ipynb

Initial protein concentrations: Get expression levels: uses data from [GSE102475](https://github.com/NTUMitoLab/RetroSignalModel.jl?organization=NTUMitoLab&organization=NTUMitoLab#differential-gene-expression).

In [ ]:
idxmulS = psmap[rs.mul_S]

r3nc_wt = map(0.0:0.02:1.0) do s
    p = copy(prob.p)
    p[idxmulS] = s
    sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
    rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
end

In [ ]:
plot(0.0:0.02:1.0, r3nc_wt, label="Wild Type", lw=2)

idxBmh = psmap[rs.ΣBmh]

for i in 1:6
    r3nc_Δb = map(0.0:0.02:1.0) do s
        p = copy(prob.p)
        p[idxmulS] = s
        p[idxBmh] = prob.p[idxBmh] / 10^i
        sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
        rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
    end

    plot!(0.0:0.02:1.0, r3nc_Δb, label = string("Bmh ", L"10^{-%$i}", " X"))
end

plot!(legend=:right, title="Bmh Knockdown", xlabel="Damage Signal", ylabel="Ratio of nucleus to cytosol RTG3", xlims=(0.0, 1.0), ylims=(0.0, 5.0))

## Mks partial knockdown

In [ ]:
idxΣMks = psmap[rs.ΣMks]

plot(0.0:0.02:1.0, r3nc_wt, label="Wild Type", lw=2)

for i in 1:5
    r3nc_Δb = map(0.0:0.02:1.0) do s
        p = copy(prob.p)
        p[idxmulS] = s
        p[idxBmh] = prob.p[idxΣMks] / 10^i
        sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
        rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
    end

    plot!(0.0:0.02:1.0, r3nc_Δb, label = string("Mks ", L"10^{-%$i}", " X"))
end

plot!(legend=:right, title="Mks Knockdown", xlabel="Damage Signal", ylabel="Ratio of nucleus to cytosol RTG3", xlims=(0.0, 1.0), ylims=(0.0, 5.0))

## Rapamycin and glutamate

`kn2M` was decreased ten times to simulated enhanced binding of Mks and Rtg2 upon Rapamycin / glutamate addition.

In [ ]:
idxkn2M = psmap[rs.kn2M]
r3nc_ra = map(0.0:0.02:1.0) do s
    p = copy(prob.p)
    p[idxmulS] = s
    p[idxkn2M] /= 10
    sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
    rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
end

pl1 = plot(0.0:0.02:1.0, [r3nc_wt r3nc_ra], label=["Wild Type" "Rapamycin / Glutamate"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 5.0),  xlabel="Damage Signal", ylabel="Ratio of nucleus to cytosol RTG3")

### Interactions with protein knockout

In [ ]:
idxmulS = psmap[rs.mul_S]
idxBmh = psmap[rs.ΣBmh]
idxΣMks = psmap[rs.ΣMks]
idxkn2M = psmap[rs.kn2M]
idxΣRtg1 = psmap[rs.ΣRtg1]
idxΣRtg2 = psmap[rs.ΣRtg2]
idxΣRtg3 = psmap[rs.ΣRtg3]

function remake_rapa(prob)
    p = copy(prob.p)
    p[idxkn2M] /= 10
    remake(prob; p = p)
end

function remake_ΔBmh(prob)
    p = copy(prob.p)
    p[idxBmh] *= 1e-4
    remake(prob; p = p)
end

function remake_ΔMks(prob)
    p = copy(prob.p)
    p[idxΣMks] *= 1e-4
    remake(prob; p = p)
end

function remake_ΔRtg1(prob)
    p = copy(prob.p)
    p[idxΣRtg1] *= 1e-4
    remake(prob; p = p)
end

function remake_ΔRtg2(prob)
    p = copy(prob.p)
    p[idxΣRtg2] *= 1e-4
    remake(prob; p = p)
end

function remake_ΔRtg3(prob)
    p = copy(prob.p)
    p[idxΣRtg3] *= 1e-4
    remake(prob; p = p)
end

function get_r3nc(s, prob)
    p = copy(prob.p)
    p[idxmulS] = s
    sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
    return rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
end

In [ ]:
f_r3nc_wt(s) = get_r3nc(s, prob)
f_r3nc_ra(s) = get_r3nc(s, prob |> remake_rapa)
f_r3nc_ΔBmh(s) = get_r3nc(s, prob |> remake_ΔBmh)
f_r3nc_ΔBmh_ra(s) = get_r3nc(s, prob |> remake_ΔBmh |> remake_rapa)
f_r3nc_ΔMks(s) = get_r3nc(s, prob |> remake_ΔMks)
f_r3nc_ΔMks_ra(s) = get_r3nc(s, prob |> remake_ΔMks |> remake_rapa)
f_r3nc_ΔRtg1(s) = get_r3nc(s, prob |> remake_ΔRtg1)
f_r3nc_ΔRtg1_ra(s) = get_r3nc(s, prob |> remake_ΔRtg1 |> remake_rapa)
f_r3nc_ΔRtg2(s) = get_r3nc(s, prob |> remake_ΔRtg2)
f_r3nc_ΔRtg2_ra(s) = get_r3nc(s, prob |> remake_ΔRtg2 |> remake_rapa)
f_r3nc_ΔRtg3(s) = get_r3nc(s, prob |> remake_ΔRtg3)
f_r3nc_ΔRtg3_ra(s) = get_r3nc(s, prob |> remake_ΔRtg3 |> remake_rapa)

In [ ]:
pl1 = plot(
    [f_r3nc_wt f_r3nc_ra], 0.0, 1.0, 
    label=["WT" "Rapa"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 5.0),  
    xlabel="Damage", ylabel="nucleus:cytosol RTG3",
    title = "Without knockout")

pl2 = plot(
    [f_r3nc_ΔBmh f_r3nc_ΔBmh_ra], 0.0, 1.0, 
    label=["WT" "Rapa"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 5.0),  
    xlabel="Damage", ylabel="nucleus:cytosol RTG3",
    title = "Bmh knockout")

pl3 = plot(
    [f_r3nc_ΔMks f_r3nc_ΔMks_ra], 0.0, 1.0, 
    label=["WT" "Rapa"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 5.0),  
    xlabel="Damage", ylabel="nucleus:cytosol RTG3",
    title = "Mks knockout")

pl4 = plot(
    [f_r3nc_ΔRtg1 f_r3nc_ΔRtg1_ra], 0.0, 1.0, 
    label=["WT" "Rapa"], lw=2, legend=:right, 
    # xlims=(0.0, 1.0), ylims=(0.0, 5.0),  
    xlabel="Damage", ylabel="nucleus:cytosol RTG3",
    title = "Rtg1 knockout")

pl5 = plot(
    [f_r3nc_ΔRtg2 f_r3nc_ΔRtg2_ra], 0.0, 1.0, 
    label=["WT" "Rapa"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 1.0),  
    xlabel="Damage", ylabel="nucleus:cytosol RTG3",
    title = "Rtg2 knockout")

pl6 = plot(
    [f_r3nc_ΔRtg3 f_r3nc_ΔRtg3_ra], 0.0, 1.0, 
    label=["WT" "Rapa"], lw=2, legend=:right, 
    # xlims=(0.0, 1.0), ylims=(0.0, 5.0),  
    xlabel="Damage", ylabel="nucleus:cytosol RTG3",
    title = "Rtg3 knockout")

plot(pl1, pl2, pl3, pl4, pl5, pl6, layout = (3, 2), size=(600, 800), left_margin = 5mm)

## Gene knockout and Rtg 3 distribution

In [ ]:
colors = [:black :red :orange :purple :green :blue]
labels = ["WT" "ΔBmh" "ΔMks" "ΔRtg1" "ΔRtg2" "ΔRtg3"]
labels_rapa = labels .* " (Rapa)"

In [ ]:
plot([f_r3nc_wt f_r3nc_ΔBmh f_r3nc_ΔMks f_r3nc_ΔRtg1 f_r3nc_ΔRtg2 f_r3nc_ΔRtg3], 0.0, 1.0,
label=labels, linestyle=:solid, linecolor=colors,
xlabel="Damage", ylabel="Nucleus:cytosol RTG3")

plot!([f_r3nc_ra f_r3nc_ΔBmh_ra f_r3nc_ΔMks_ra f_r3nc_ΔRtg1_ra f_r3nc_ΔRtg2_ra f_r3nc_ΔRtg3_ra], 0.0, 1.0,
label = labels_rapa, linestyle=:dash, linecolor = colors,
xlabel="Damage", ylabel="Nucleus:cytosol RTG3", size=(600, 600))